In [ ]:
# 

In [92]:
import zipfile
import re
import os
import numpy as np
import pandas as pd

1. Reading the data in with pd.read_csv . IIRC use delimiter=| and encoding=latin_1
2. Merge the speeches (speeches_xxx) with the metadata (descr_xxx) and speaker metadata (speakermap)
3. Convert dates from 19890103 format to something more reasonable (eg datetime 1989-01-03)
4. Filter out independents

### Unzip

In [ ]:
resource_path = "resources"
file_name = 'hein-daily.zip'
file_path = os.path.join(resource_path, file_name)

if re.search(r'\.zip$', file_name):
    with zipfile.ZipFile(file_path,"r") as zip_ref:
        zip_ref.extractall(resource_path)

### View content

In [26]:
def view_content(file, path = 'resources/hein-daily/' ):
    p = os.path.join(path, file)
    with open(p) as f:
        content = f.readlines()

    print('Num of lines:', len(content))
    print(content[0:20])


In [27]:
view_content('097_SpeakerMap.txt')

Num of lines: 202055
['speakerid|speech_id|lastname|firstname|chamber|state|gender|party|district|nonvoting\n', '97105301|970000005|PERCY|CHARLES|S|IL|M|R||voting\n', '97105071|970000006|HUDDLESTON|WALTER|S|KY|M|D||voting\n', '97106761|970000007|JACKSON|HENRY|S|WA|M|D||voting\n', '97106981|970000008|STAFFORD|ROBERT|S|VT|M|R||voting\n', '97104811|970000010|BAKER|HOWARD|S|TN|M|R||voting\n', '97104811|970000012|BAKER|HOWARD|S|TN|M|R||voting\n', '97107681|970000013|CRANSTON|ALAN|S|CA|M|D||voting\n', '97104811|970000015|BAKER|HOWARD|S|TN|M|R||voting\n', '97104811|970000016|BAKER|HOWARD|S|TN|M|R||voting\n', '97104811|970000019|BAKER|HOWARD|S|TN|M|R||voting\n', '97104811|970000021|BAKER|HOWARD|S|TN|M|R||voting\n', '97104811|970000023|BAKER|HOWARD|S|TN|M|R||voting\n', '97107391|970000026|MCCLURE|JAMES|S|ID|M|R||voting\n', '97104811|970000028|BAKER|HOWARD|S|TN|M|R||voting\n', '97106241|970000029|MATHIAS|CHARLES|S|MD|M|R||voting\n', '97104811|970000033|BAKER|HOWARD|S|TN|M|R||voting\n', '97104811

In [28]:
view_content('descr_097.txt')

Num of lines: 283399
['speech_id|chamber|date|number_within_file|speaker|first_name|last_name|state|gender|line_start|line_end|file|char_count|word_count\n', '970000001|S|19810105|00001|The VICE PRESIDENT|Unknown|Unknown|Unknown|Special|000017|000062|01051981.txt|1543|283\n', '970000002|S|19810105|00002|The VICE PRESIDENT|Unknown|Unknown|Unknown|Special|000065|000074|01051981.txt|334|55\n', '970000003|S|19810105|00003|The VICE|Unknown|Unknown|Unknown|Special|000200|000212|01051981.txt|466|82\n', '970000004|S|19810105|00004|The VICE PRESIDENT|Unknown|Unknown|Unknown|Special|000858|000864|01051981.txt|210|37\n', '970000005|S|19810105|00005|Mr. PERCY|Unknown|PERCY|Unknown|M|000883|000889|01051981.txt|201|34\n', '970000006|S|19810105|00006|Mr. HUDDLESTON|Unknown|HUDDLESTON|Unknown|M|000895|000901|01051981.txt|202|34\n', '970000007|S|19810105|00007|Mr. JAcKSON|Unknown|JACKSON|Unknown|M|000907|000913|01051981.txt|201|34\n', '970000008|S|19810105|00008|Mr. STAFFORD|Unknown|STAFFORD|Unknown|M|

In [29]:
view_content('speeches_097.txt')

Num of lines: 283399
['speech_id|speech\n', '970000001|The Senate vill please come to order. The Chaplain will offer the prayer. The Chaplain. the Reverend Edward L. R. Elson. D.D.. offered the following prayer: PRAYER God of our fathers and our God. who has made and preserved us a nation. we thank Thee for Thy care over us from generation to generation. and for the new era which unfolds before us. Let Thy spirit descend upon us with all Its cleansing. healing. and creative power. Make usnew persons for new times in a land newborn in spirit. May the solemn induction of the new Members of this body be a renewal of the vocatinal vows of all others. In the deliberations of this new Congress keep our motives pure. our vision clear. our patriotism undefiled. our speech chaste. our judgments fair. and our consciences unbetrayed. Help us to develop work patterns which sustain the soul. nourish the mind. and strengthen the will. And give us grace to make time for the family. for home. and for 

### Some analysis

In [80]:
# Issue with random "|"" scattered into the speeches -> error_bad_lines=False. And remove those speeches from descr

path = 'resources/hein-daily/' 
display(pd.read_csv(os.path.join(path, '109_SpeakerMap.txt'), sep='|', encoding='latin_1').head())
display(pd.read_csv(os.path.join(path, 'descr_109.txt'), sep='|', encoding='latin_1').head())
display(pd.read_csv(os.path.join(path, 'speeches_109.txt'), sep='|', encoding='latin_1', error_bad_lines=False ).head())

,speakerid,speech_id,lastname,firstname,chamber,state,gender,party,district,nonvoting
0,109112991,1090000002,FRIST,WILLIAM,S,TN,M,R,NaN,voting
1,109118161,1090000006,FEINSTEIN,DIANNE,S,CA,F,D,NaN,voting
2,109116411,1090000013,AKAKA,DANIEL,S,HI,M,D,NaN,voting
3,109117451,1090000016,PRYOR,MARK,S,AR,M,D,NaN,voting
4,109113281,1090000019,SARBANES,PAUL,S,MD,M,D,NaN,voting


,speech_id,chamber,date,number_within_file,speaker,first_name,last_name,state,gender,line_start,line_end,file,char_count,word_count
0,1090000001,S,20050104,1,The VICE PRESIDENT,Unknown,Unknown,Unknown,Special,60,62,01042005.txt,34,5
1,1090000002,S,20050104,2,Mr. FRIST,Unknown,FRIST,Unknown,M,65,78,01042005.txt,493,90
2,1090000003,S,20050104,3,The VICE PRESIDENT,Unknown,Unknown,Unknown,Special,79,82,01042005.txt,102,17
3,1090000004,S,20050104,4,The VICE PRESIDENT,Unknown,Unknown,Unknown,Special,85,96,01042005.txt,426,73
4,1090000005,S,20050104,5,The VICE PRESIDENT,Unknown,Unknown,Unknown,Special,776,784,01042005.txt,230,43


C:\Users\isacb\AppData\Local\Temp\ipykernel_3968\540555285.py:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  display(pd.read_csv(os.path.join(path, 'speeches_109.txt'), sep='|', encoding='latin_1', error_bad_lines=False ).head())
Skipping line 7151: expected 2 fields, saw 3
Skipping line 26137: expected 2 fields, saw 3
Skipping line 32834: expected 2 fields, saw 3



,speech_id,speech
0,1090000001,The majority leader is recognized.
1,1090000002,Mr. President. these last 9 days have brought ...
2,1090000003,Without objection. it is so ordered. The Senat...
3,1090000004,The Chair lays before the Senate the certifica...
4,1090000005,If the Senators to be sworn will now present t...


In [75]:
p = os.path.join(path, 'speeches_109.txt')
with open(p) as f:
    content = f.readlines()

print(len(content[7150].split('|')))
print(content[7150].split('|'))
print()
print(content[7150])


3
['1090007150', 'Mr. Speaker. it gives me a great deal of pleasure to recognize Worshipful Master R.W. William J. Ewing for his leadership of Jephthah Lodge No. 56 in Montclair. New Jersey. Under Worshipful Master Ewings leadership. overall membership increased and acknowledgments of community initiatives improved. His unwavering dedication and passion have inspired the brothers of Jephthah Lodge No. 56 to increase their own commitments to do more within their designated communities. Worshipful Master Ewing performed his duties as caretaker of the East. not only by word. but more importantly. through his deeds. He has made inkind donations by extending the use of his business office. including his office manager and staff. consistently throughout his tenure. He has also made substantial financial contributions to assist the lodge. These unselfish acts have contributed significantly to the success of the lodge. In addition to his lodge activities. Worshipful Master Ewing is a good citi

In [81]:
# Check if ID is unique
X = pd.read_csv(os.path.join(path, 'speeches_109.txt'), sep='|', encoding='latin_1', error_bad_lines=False )
print('Len X:',len(X))
print('N Unique speech ID:', len(X['speech_id'].unique()))

C:\Users\isacb\AppData\Local\Temp\ipykernel_3968\3945375732.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  X = pd.read_csv(os.path.join(path, 'speeches_109.txt'), sep='|', encoding='latin_1', error_bad_lines=False )
Skipping line 7151: expected 2 fields, saw 3
Skipping line 26137: expected 2 fields, saw 3
Skipping line 32834: expected 2 fields, saw 3



Len X: 187431
N Unique speech ID: 187431


In [85]:
# Speaker ID? 
X = pd.read_csv(os.path.join(path, 'descr_109.txt'), sep='|', encoding='latin_1' )
print('Any null Speaker?', X['speaker'].isnull().any())

Any null Speaker? False


In [88]:
# Speaker
X = pd.read_csv(os.path.join(path, '109_SpeakerMap.txt'), sep='|', encoding='latin_1' )
X['party'].unique() #Republic, Demo, Independent

array(['R', 'D', 'I'], dtype=object)

# 

In [96]:

S = pd.read_csv(os.path.join(path, 'speeches_109.txt'), sep='|', encoding='latin_1', on_bad_lines='skip' )
D = pd.read_csv(os.path.join(path, 'descr_109.txt'), sep='|', encoding='latin_1' )
SM = pd.read_csv(os.path.join(path, '109_SpeakerMap.txt'), sep='|', encoding='latin_1' )


In [102]:
X = S.merge(D, on='speech_id').merge(SM, on='speech_id')

In [105]:
X

,speech_id,speech,chamber_x,date,number_within_file,speaker,first_name,last_name,state_x,gender_x,...,word_count,speakerid,lastname,firstname,chamber_y,state_y,gender_y,party,district,nonvoting
0,1090000002,Mr. President. these last 9 days have brought ...,S,20050104,2,Mr. FRIST,Unknown,FRIST,Unknown,M,...,90,109112991,FRIST,WILLIAM,S,TN,M,R,NaN,voting
1,1090000006,respectively. advanced to the desk of the Vice...,S,20050104,6,Mrs. FEINSTEIN,Unknown,FEINSTEIN,Unknown,F,...,34,109118161,FEINSTEIN,DIANNE,S,CA,F,D,NaN,voting
2,1090000013,respectively. advanced to the desk of the Vice...,S,20050104,13,Mr. AKAKA,Unknown,AKAKA,Unknown,M,...,34,109116411,AKAKA,DANIEL,S,HI,M,D,NaN,voting
3,1090000016,former Senator Hawkins and former Senator Broc...,S,20050104,16,Mr. PRYOR,Unknown,PRYOR,Unknown,M,...,41,109117451,PRYOR,MARK,S,AR,M,D,NaN,voting
4,1090000019,Mr. STEVENS. former Senator Murkowski. and Ms....,S,20050104,19,Mr. SARBANES,Unknown,SARBANES,Unknown,M,...,42,109113281,SARBANES,PAUL,S,MD,M,D,NaN,voting
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116995,1090187563,Madam Speaker. Congressman MIKE OXLEY served t...,H,20061208,1798,Mr. MCHENRY,Unknown,MCHENRY,Unknown,M,...,145,109120460,MCHENRY,PATRICK,H,NC,M,R,10.0,voting
116996,1090187564,Mr. Speaker. I come before this body today as ...,H,20061208,1799,Ms. MCKINNEY,Unknown,MCKINNEY,Unknown,F,...,705,109114340,MCKINNEY,CYNTHIA,H,GA,F,D,4.0,voting
116997,1090187565,Mr. Speaker. it has been an honor and privileg...,H,20061208,1800,Mr. BUYER,Unknown,BUYER,Unknown,M,...,218,109115420,BUYER,STEPHEN,H,IN,M,R,4.0,voting
116998,1090187566,Mr. Speaker. I rise with a very heavy heart an...,H,20061208,1801,Mrs. MUSGRAVE,Unknown,MUSGRAVE,Unknown,F,...,854,109115070,MUSGRAVE,MARILYN,H,CO,F,R,4.0,voting


In [101]:
SM

,speakerid,speech_id,lastname,firstname,chamber,state,gender,party,district,nonvoting
0,109112991,1090000002,FRIST,WILLIAM,S,TN,M,R,NaN,voting
1,109118161,1090000006,FEINSTEIN,DIANNE,S,CA,F,D,NaN,voting
2,109116411,1090000013,AKAKA,DANIEL,S,HI,M,D,NaN,voting
3,109117451,1090000016,PRYOR,MARK,S,AR,M,D,NaN,voting
4,109113281,1090000019,SARBANES,PAUL,S,MD,M,D,NaN,voting
...,...,...,...,...,...,...,...,...,...,...
117126,109121030,1090179744,ROGERS,HAROLD,H,KY,M,R,5.0,voting
117127,109116930,1090182461,LEWIS,JERRY,H,CA,M,R,41.0,voting
117128,109114700,1090062389,DAVIS,THOMAS,H,VA,M,R,11.0,voting
117129,109119950,1090068205,JOHNSON,EDDIE,H,TX,F,D,30.0,voting


In [100]:
D

,speech_id,chamber,date,number_within_file,speaker,first_name,last_name,state,gender,line_start,line_end,file,char_count,word_count
0,1090000001,S,20050104,1,The VICE PRESIDENT,Unknown,Unknown,Unknown,Special,60,62,01042005.txt,34,5
1,1090000002,S,20050104,2,Mr. FRIST,Unknown,FRIST,Unknown,M,65,78,01042005.txt,493,90
2,1090000003,S,20050104,3,The VICE PRESIDENT,Unknown,Unknown,Unknown,Special,79,82,01042005.txt,102,17
3,1090000004,S,20050104,4,The VICE PRESIDENT,Unknown,Unknown,Unknown,Special,85,96,01042005.txt,426,73
4,1090000005,S,20050104,5,The VICE PRESIDENT,Unknown,Unknown,Unknown,Special,776,784,01042005.txt,230,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187641,1090187642,E,20061208,1877,Ms. LEE,Unknown,LEE,Unknown,F,187444,187486,12082006.txt,1673,301
187642,1090187643,E,20061208,1878,Mr. SOUDER,Unknown,SOUDER,Unknown,M,187495,187635,12082006.txt,5336,867
187643,1090187644,E,20061208,1879,Mr. INGLIS of South Carolina,Unknown,INGLIS,South Carolina,M,187641,187644,12082006.txt,106,19
187644,1090187645,E,20061208,1880,Ms. WOOLSEY,Unknown,WOOLSEY,Unknown,F,187651,187681,12082006.txt,950,160


In [104]:
S

,speech_id,speech
0,1090000001,The majority leader is recognized.
1,1090000002,Mr. President. these last 9 days have brought ...
2,1090000003,Without objection. it is so ordered. The Senat...
3,1090000004,The Chair lays before the Senate the certifica...
4,1090000005,If the Senators to be sworn will now present t...
...,...,...
187426,1090187642,Mr. Speaker. I rise in opposition to the under...
187427,1090187643,Madam Speaker. yesterday. we passed H.R. 6344....
187428,1090187644,Mr. Speaker. on rollcall No. 540 I was unavoid...
187429,1090187645,Mr. Speaker. family abductions are the most co...


In [144]:
from datetime import datetime

class DatasetHeinDaily:

    def __init__(self, directory, drop_independent=True):
        self.directory = directory
        
        self.descr:dict = get_file_names('descr', self.directory) #id : filename
        self.speeches:dict = get_file_names('speeches', self.directory)
        self.speaker_map:dict = get_file_names('SpeakerMap', self.directory)
        assert not set(self.descr.keys()).symmetric_difference(self.speeches.keys()) and not set(self.descr.keys()).symmetric_difference(self.speaker_map.keys()), 'The ids of the files do not overlap'
        
        self.drop_independent = drop_independent

        self.track_sizes = pd.DataFrame(columns=['speeches', 'after_merge', 'after_drop_independent']) # for verbosity

    def preprocess(self):

        pd_to_concat = []
        for i in self.descr.keys():
            S = pd.read_csv(os.path.join(self.directory, self.speeches[i]), sep='|', encoding='latin_1', on_bad_lines='skip')
            D = pd.read_csv(os.path.join(self.directory, self.descr[i]), sep='|', encoding='latin_1', \
                             parse_dates=['date'], date_parser=lambda x: datetime.strptime(x, '%Y%m%d'))
            SM = pd.read_csv(os.path.join(self.directory, self.speaker_map[i]), sep='|', encoding='latin_1')
            
            self.track_sizes.loc[i, 'speeches'] = len(S)
            
            X = S.merge(D, on='speech_id').merge(SM, on='speech_id')
            self.track_sizes.loc[i, 'after_merge'] = len(X)

            if self.drop_independent:
                X = X[X['party']!='I']
                self.track_sizes.loc[i, 'after_drop_independent'] = len(X)

            pd_to_concat.append(X)

        return pd.concat(pd_to_concat, axis=0, ignore_index=True)
            
def get_file_names(file_category, directory)->dict:
    # ex: file_naming = 'speeches'
    out = {} # digit_id: file_name  (speeches_110.txt -> digit_id = 110 )
    for file_name in os.listdir(directory):
        rgx = r'^.*%s.*$'%(file_category)
        if re.search(rgx, file_name):
            id = re.findall(r'\d+', file_name)
            assert len(id) == 1, f'Ambigious/No digit IDs found for {file_name}. Found: {id}'
            assert id[0] not in out, f'{id[0]} exists multiple times'
            out[id[0]] = file_name
    return out

In [145]:
path = 'resources/hein-daily/' 

In [146]:
ds = DatasetHeinDaily(directory=path)
dataset = ds.preprocess()

In [151]:
ds.track_sizes.sum(axis=0)['speeches']

3862028

### Take a look at processed data

In [154]:
X = pd.read_csv('hein_daily.csv')

In [155]:
X.head()

,speech_id,speech,chamber_x,date,number_within_file,speaker,first_name,last_name,state_x,gender_x,...,word_count,speakerid,lastname,firstname,chamber_y,state_y,gender_y,party,district,nonvoting
0,970000005,respectively. advanced to the desk of the Vive...,S,1981-01-05,5,Mr. PERCY,Unknown,PERCY,Unknown,M,...,34,97105301,PERCY,CHARLES,S,IL,M,R,NaN,voting
1,970000006,respectively. advanced to the desk of the Vice...,S,1981-01-05,6,Mr. HUDDLESTON,Unknown,HUDDLESTON,Unknown,M,...,34,97105071,HUDDLESTON,WALTER,S,KY,M,D,NaN,voting
2,970000007,respectively. advanced to the desk of the Vice...,S,1981-01-05,7,Mr. JAcKSON,Unknown,JACKSON,Unknown,M,...,34,97106761,JACKSON,HENRY,S,WA,M,D,NaN,voting
3,970000008,respectively. advanced to the desk of the Vice...,S,1981-01-05,8,Mr. STAFFORD,Unknown,STAFFORD,Unknown,M,...,34,97106981,STAFFORD,ROBERT,S,VT,M,R,NaN,voting
4,970000010,Mr. President. I suggest the absence of a quorum.,S,1981-01-05,10,Mr. BAKER,Unknown,BAKER,Unknown,M,...,9,97104811,BAKER,HOWARD,S,TN,M,R,NaN,voting


In [156]:
X['speech_id'].nunique()

2576030

In [157]:
len(X)

2576030